In [1]:
import pandas as pd
import sqlite3
sqlite_db = 'database.sqlite'
conn = sqlite3.connect(sqlite_db)
c = conn.cursor()

In [2]:
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)

In [3]:
loans = pd.read_csv('loan.csv')
loans.head()

/Users/polbrigneti/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
0,1077501,1296599,5000.0,5000.0,4975.0,36 months,10.65,162.87,B,B2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1077430,1314167,2500.0,2500.0,2500.0,60 months,15.27,59.83,C,C4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1077175,1313524,2400.0,2400.0,2400.0,36 months,15.96,84.33,C,C5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1076863,1277178,10000.0,10000.0,10000.0,36 months,13.49,339.31,C,C1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1075358,1311748,3000.0,3000.0,3000.0,60 months,12.69,67.79,B,B5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
loans['length_years'] = loans.loan_term/12

In [13]:
loans.year = loans.year.astype(int)

In [14]:
loans['end_d'] = loans.year + loans.length_years

In [15]:
loans['end_date'] = loans.month +"-"+loans.end_d.map(str)

In [6]:
# Extracting month, year and loan_term
loans['month'] = loans.issue_d.apply(lambda x: x[:3])
loans['year'] = loans.issue_d.apply(lambda x: x[4:])
loans['loan_term'] = loans.term.apply(lambda x: x[:3])
loans.loan_term = loans.loan_term.astype('int64')

In [7]:
def month_num(input_str):
    if input_str == 'Jan':
        return '01'
    elif input_str == 'Feb':
        return '02'
    elif input_str == 'Mar':
        return '03'
    elif input_str == 'Apr':
        return '04'
    elif input_str == 'May':
        return '05'
    elif input_str == 'Jun':
        return '06'
    elif input_str == 'Jul':
        return '07'
    elif input_str == 'Aug':
        return '08'
    elif input_str == 'Sep':
        return '09'
    elif input_str == 'Oct':
        return '10'
    elif input_str == 'Nov':
        return '11'
    elif input_str == 'Dec':
        return '12'

In [8]:
def quarter_num(input_str):
    if input_str == 'Jan':
        return 'Q1'
    elif input_str == 'Feb':
        return 'Q1'
    elif input_str == 'Mar':
        return 'Q1'
    elif input_str == 'Apr':
        return 'Q2'
    elif input_str == 'May':
        return 'Q2'
    elif input_str == 'Jun':
        return 'Q2'
    elif input_str == 'Jul':
        return 'Q3'
    elif input_str == 'Aug':
        return 'Q3'
    elif input_str == 'Sep':
        return 'Q3'
    elif input_str == 'Oct':
        return 'Q4'
    elif input_str == 'Nov':
        return 'Q4'
    elif input_str == 'Dec':
        return 'Q4'

In [9]:
# Adding columns with month number and quarter
loans['month_num'] = loans.month.apply(lambda x: month_num(x))
loans['quarter'] = loans.month.apply(lambda x: quarter_num(x))

In [10]:
# Calculating loans revenue and gmv
loans['loan_gmv'] = loans.loan_term * loans.installment
loans['loan_revenue'] = loans.loan_gmv - loans.loan_amnt

In [ ]:
% default rate
% active loans
loans per month
installment per default rate

In [111]:
loans_db = loans[['id','member_id','loan_amnt','loan_term','funded_amnt','funded_amnt_inv','grade', 'purpose','sub_grade','addr_state','int_rate','issue_d','installment','month','term','year','loan_status','month_num','quarter','loan_revenue','loan_gmv','total_pymnt']]
loans_db.to_sql('loans_data', con=engine)

### Distribution of the loans

### Outstanding loans over time

In [126]:
outstanding_loans = engine.execute("""SELECT 
                                        year, 
                                        quarter, 
                                        COUNT(id) 
                                    FROM 
                                        loan
                                    WHERE 
                                        loan_status <> "Fully paid" 
                                    GROUP BY 
                                        1,2
                                    ORDER BY
                                        1""").fetchall()
outstanding_loans_df = pd.DataFrame(data=outstanding_loans, columns=['year','quarter','loans'])
outstanding_loans_df

,year,quarter,loans
0,2007,Q2,24
1,2007,Q3,190
2,2007,Q4,389
3,2008,Q1,1013
4,2008,Q2,498
5,2008,Q3,298
6,2008,Q4,584
7,2009,Q1,895
8,2009,Q2,1098
9,2009,Q3,1364


In [121]:
outstanding_loans_df.to_csv('outstanding_loans')

In [127]:
amnt_outstanding = engine.execute("""
                                    SELECT 
                                        year, 
                                        quarter, 
                                        SUM(loan_amnt) 
                                    FROM 
                                        loan
                                    WHERE 
                                        loan_status <> "Fully paid" 
                                    GROUP BY 
                                        1,2
                                    ORDER BY
                                        1""").fetchall()
amnt_outstanding_df = pd.DataFrame(data=amnt_outstanding, columns=['year','quarter','loans'])
amnt_outstanding_df

,year,quarter,loans
0,2007,Q2,9.185000e+04
1,2007,Q3,1.236575e+06
2,2007,Q4,3.649050e+06
3,2008,Q1,1.003528e+07
4,2008,Q2,3.722950e+06
5,2008,Q3,1.766600e+06
6,2008,Q4,5.594425e+06
7,2009,Q1,8.246350e+06
8,2009,Q2,1.011508e+07
9,2009,Q3,1.330202e+07


In [129]:
amnt_outstanding_df.to_csv('amnt_outstanding_loans')

### Current status of the loans

In [92]:
loans.loan_status.value_counts()

Current                                                601779
Fully Paid                                             207723
Charged Off                                             45248
Late (31-120 days)                                      11591
Issued                                                   8460
In Grace Period                                          6253
Late (16-30 days)                                        2357
Does not meet the credit policy. Status:Fully Paid       1988
Default                                                  1219
Does not meet the credit policy. Status:Charged Off       761
Name: loan_status, dtype: int64

### Grade classifications of the loans

In [ ]:
# Loans per grade
loans_grade = loans.grade.value_counts()
loans_grade.to_csv('loans_grade')

In [ ]:
loans_grade

In [ ]:
loans.groupby(['grade','sub_grade'])['loan_amnt'].sum()

In [ ]:
# Amount borrowed per grade 
amnt_per_grade = engine.execute("""SELECT 
                                        grade, 
                                        SUM(loan_amnt) 
                                    FROM 
                                        loan 
                                    GROUP BY 
                                        1""").fetchall()
amnt_per_grade

In [ ]:
# Loans per sub_grade
loans.sub_grade.value_counts()

In [ ]:
# Amount borrowed per sub_grade 
amnt_per_sub_grade = engine.execute(""""SELECT 
                                            sub_grade, 
                                            SUM(loan_amnt) 
                                        FROM 
                                            loan 
                                        GROUP BY 
                                            1""").fetchall()
amnt_per_sub_grade_df = pd.DataFrame(data=amnt_per_sub_grade, columns=['sub_grade','loan_amount'])
amnt_per_sub_grade_df.to_csv('amnt_per_sub_grade')

### Loan Purposes

In [ ]:
loans.purpose.value_counts()

In [ ]:
amnt_loans_purpose = engine.execute("""SELECT 
                                            purpose, 
                                            SUM(loan_amnt) 
                                        FROM 
                                            loan_data 
                                        GROUP BY 
                                            1 
                                        ORDER BY 
                                            2 DESC""").fetchall()
amnt_loans_purpose_df = pd.DataFrame(data=amount_loans_purpose, columns=['purpose','loan_amount'])
amnt_loans_purpose_df.to_csv('amnt_loans_purpose')

### Distribution of loans per state (Volume and Value)

In [ ]:
loan_state_distribution = engine.execute("""SELECT 
                                                addr_state, 
                                                SUM(loan_amnt) AS value, 
                                                COUNT(addr_state) AS volume 
                                            FROM 
                                                loans_data 
                                            GROUP BY 
                                                1""").fetchall()
loan_state_distribution_df = pd.DataFrame(data=loan_state_distribution, columns=['addr_state','value', 'volume'])

In [ ]:
loan_state_distribution_df.to_csv('loan_state_distribution')

In [ ]:
loan_state_distribution_df

In [ ]:
loan_state_distribution_df.to_csv('loans_distribution')

### Range of Interest Rates per Grade

In [ ]:
interest_per_grade = engine.execute("""SELECT 
                                            grade, 
                                            int_rate AS interest 
                                        FROM 
                                            loan_data 
                                        GROUP BY 
                                            1,2""").fetchall()
interest_per_grade

In [ ]:
interest_per_grade_df = pd.DataFrame(data=interest_per_grade, columns=['grade','sub_grade','loans','interest'])
interest_per_grade_df.to_csv('interests_per_sub_grade')

In [65]:
interest_over_time = engine.execute("""
                                        SELECT
                                            year,
                                            month,
                                            month_num,
                                            ROUND(AVG(int_rate),2)
                                        FROM 
                                            loans
                                        WHERE
                                            grade = "G"
                                        GROUP BY
                                            1,2
                                        ORDER BY
                                            1,3,4""").fetchall()

In [66]:
interest_over_time_df = pd.DataFrame(data=interest_over_time, columns=['year','month','month_num','int_rate'])
interest_over_time_df.to_csv('interst_rate_grade')

## Trends

In [130]:
amount_loans = engine.execute('SELECT year, month, month_num, COUNT(id) FROM loan GROUP BY 1,2 ORDER BY 1,3 DESC').fetchall()

In [131]:
amount_loans

[(u'2007', u'Dec', u'12', 172),
 (u'2007', u'Nov', u'11', 112),
 (u'2007', u'Oct', u'10', 105),
 (u'2007', u'Sep', u'09', 53),
 (u'2007', u'Aug', u'08', 74),
 (u'2007', u'Jul', u'07', 63),
 (u'2007', u'Jun', u'06', 24),
 (u'2008', u'Dec', u'12', 253),
 (u'2008', u'Nov', u'11', 209),
 (u'2008', u'Oct', u'10', 122),
 (u'2008', u'Sep', u'09', 57),
 (u'2008', u'Aug', u'08', 100),
 (u'2008', u'Jul', u'07', 141),
 (u'2008', u'Jun', u'06', 124),
 (u'2008', u'May', u'05', 115),
 (u'2008', u'Apr', u'04', 259),
 (u'2008', u'Mar', u'03', 402),
 (u'2008', u'Feb', u'02', 306),
 (u'2008', u'Jan', u'01', 305),
 (u'2009', u'Dec', u'12', 658),
 (u'2009', u'Nov', u'11', 662),
 (u'2009', u'Oct', u'10', 604),
 (u'2009', u'Sep', u'09', 507),
 (u'2009', u'Aug', u'08', 446),
 (u'2009', u'Jul', u'07', 411),
 (u'2009', u'Jun', u'06', 406),
 (u'2009', u'May', u'05', 359),
 (u'2009', u'Apr', u'04', 333),
 (u'2009', u'Mar', u'03', 324),
 (u'2009', u'Feb', u'02', 302),
 (u'2009', u'Jan', u'01', 269),
 (u'2010', u'

In [132]:
amount_loans_df = pd.DataFrame(data=amount_loans, columns=['year','month','month_num','loans'])
amount_loans_df.to_csv('amount_loans')

In [81]:
loans.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,month,year,loan_term,month_num,quarter,loan_gmv,loan_revenue,length_years,end_d,end_date
0,1077501,1296599,5000.0,5000.0,4975.0,36 months,10.65,162.87,B,B2,...,Dec,2011,36,12,Q4,5863.32,863.32,3.0,2014.0,Dec-2014.0
1,1077430,1314167,2500.0,2500.0,2500.0,60 months,15.27,59.83,C,C4,...,Dec,2011,60,12,Q4,3589.80,1089.80,5.0,2016.0,Dec-2016.0
2,1077175,1313524,2400.0,2400.0,2400.0,36 months,15.96,84.33,C,C5,...,Dec,2011,36,12,Q4,3035.88,635.88,3.0,2014.0,Dec-2014.0
3,1076863,1277178,10000.0,10000.0,10000.0,36 months,13.49,339.31,C,C1,...,Dec,2011,36,12,Q4,12215.16,2215.16,3.0,2014.0,Dec-2014.0
4,1075358,1311748,3000.0,3000.0,3000.0,60 months,12.69,67.79,B,B5,...,Dec,2011,60,12,Q4,4067.40,1067.40,5.0,2016.0,Dec-2016.0


### Loan Performance

In [86]:
performance = engine.execute("""
                                SELECT
                                    grade,
                                    SUM(installment) as Issued
                                FROM
                                    loan
                                GROUP BY
                                    1""").fetchall()
performance

[(u'A', 63371052.94997727),
 (u'B', 103962094.86997947),
 (u'C', 103738680.15000223),
 (u'D', 63571180.629994854),
 (u'E', 36412625.42000008),
 (u'F', 13036069.860000035),
 (u'G', 3441903.8799999887)]

### Growth

In [44]:
growth = engine.execute("""
                        SELECT
                            year,
                            quarter,
                            COUNT(*),
                            SUM(loan_amnt)
                        FROM
                            loans
                        GROUP BY 1,2""").fetchall()
growth_df = pd.DataFrame(data=growth, columns=['year','quarter','loans','amount'])
growth_df.to_csv('growth')

### Profitability

In [155]:
# Assumption: We include the loans that "Does not meet the credit policy. Status:Fully Paid" as fully paid 
loan_profit = engine.execute("""
                SELECT
                    year,
                    month,
                    month_num,
                    ROUND(SUM(loan_amnt),2) AS amnt_borrowed,
                    ROUND(SUM(loan_gmv),2) AS return
                FROM
                    loans
                GROUP BY
                 1,2
                ORDER BY
                 1,3   """).fetchall()

loan_profit_df = pd.DataFrame(data=loan_profit, columns=['year','month','month_num','amnt_borrowed','return'])
loan_profit_df.to_csv('loan_profit')

In [156]:
loan_profit_df

,year,month,month_num,amnt_borrowed,return
0,2007,Jun,06,91850.0,1.067476e+05
1,2007,Jul,07,348325.0,3.925231e+05
2,2007,Aug,08,515300.0,5.898294e+05
3,2007,Sep,09,372950.0,4.521056e+05
4,2007,Oct,10,753225.0,8.284864e+05
5,2007,Nov,11,1008650.0,1.192374e+06
6,2007,Dec,12,1887175.0,2.187126e+06
7,2008,Jan,01,2926000.0,3.475682e+06
8,2008,Feb,02,2959225.0,3.562170e+06
9,2008,Mar,03,4150050.0,4.994212e+06


### Active loans (Value & Volume)

In [47]:
active_loans = engine.execute("""
                                SELECT
                                    year,
                                    month,
                                    month_num,
                                    COUNT(*) AS loans,
                                    SUM(loan_amnt)
                                FROM
                                    loan
                                WHERE
                                    loan_status <> "Fully Paid"
                                GROUP BY
                                    1,2
                                ORDER BY
                                    1,3""").fetchall()
active_loans_df = pd.DataFrame(data=active_loans, columns=['year','month','month_num','volume','value'])
active_loans_df.to_csv('active_loans')

In [48]:
active_loans_df

,year,month,month_num,volume,value
0,2007,Jun,06,23,84350.0
1,2007,Jul,07,33,176625.0
2,2007,Aug,08,48,357950.0
3,2007,Sep,09,38,257050.0
4,2007,Oct,10,68,504150.0
5,2007,Nov,11,82,719450.0
6,2007,Dec,12,105,1147025.0
7,2008,Jan,01,165,1531675.0
8,2008,Feb,02,157,1583800.0
9,2008,Mar,03,206,2034125.0


In [160]:
paid_loans = engine.execute("""
                                SELECT
                                    year,
                                    month,
                                    month_num,
                                    SUM(loan_amnt),
                                    ROUND(SUM(loan_gmv),2)
                                FROM
                                    loans
                                WHERE
                                    loan_status = "Fully Paid"
                                GROUP BY
                                    1,2
                                ORDER BY
                                    1,3""").fetchall()
paid_loans_df = pd.DataFrame(data=paid_loans, columns=['year','month','month_num','loan_amnt','loan_gmv'])
paid_loans_df.to_csv('paid_loans_amnt')

In [40]:
loans.loan_status.value_counts()

Current                                                601779
Fully Paid                                             207723
Charged Off                                             45248
Late (31-120 days)                                      11591
Issued                                                   8460
In Grace Period                                          6253
Late (16-30 days)                                        2357
Does not meet the credit policy. Status:Fully Paid       1988
Default                                                  1219
Does not meet the credit policy. Status:Charged Off       761
Name: loan_status, dtype: int64

### Grade distribution per year

In [122]:
grade_distribution = engine.execute("""
                SELECT
                    year,
                    grade,
                    COUNT(*) as default_loans
                FROM
                    loans_data
                WHERE
                    loan_term = "36"
                GROUP BY
                    1,2""").fetchall()
grade_distribution_df = pd.DataFrame(data = grade_distribution, columns=['year','grade','loans'])
grade_distribution_df.to_csv('grade_distribution')

In [123]:
grade_distribution_df

,year,grade,loans
0,2007,A,78
1,2007,B,98
2,2007,C,141
3,2007,D,99
4,2007,E,100
5,2007,F,52
6,2007,G,35
7,2008,A,318
8,2008,B,594
9,2008,C,580


In [96]:
loans.term.value_counts()

 36 months    621125
 60 months    266254
Name: term, dtype: int64

### Default rate

In [90]:
default_rate = engine.execute("""
                SELECT
                    year,
                    month,
                    month_num,
                    SUM(CASE WHEN loan_status IN("Charged Off","Default","Does not meet the credit policy. Status:Charged Off ") THEN 1 ELSE 0 END) as default_loans,
                    COUNT(*) as total_loans
                FROM
                    loan
                GROUP BY
                    1,2
                ORDER BY
                    1,3""").fetchall()
default_rate_df = pd.DataFrame(data=default_rate, columns=['year','month','month_num','default_loans','total_loans'])
default_rate_df.to_csv('default_rate')

In [23]:
amnt_default_rate = engine.execute("""
                SELECT
                    year,
                    month,
                    month_num,
                    SUM(CASE WHEN loan_status IN("Charged Off","Default","Does not meet the credit policy. Status:Charged Off ") THEN 1 ELSE 0 END) as default_loans,
                    ROUND(SUM(total_pymnt),2) as volume,
                    SUM(loan_amnt)
                FROM
                    loans
                WHERE
                    loan_status IN("Charged Off","Default","Does not meet the credit policy. Status:Charged Off ") AND grade = "A"
                GROUP BY
                    1,2
                ORDER BY
                    1,3""").fetchall()
amnt_default_rate_df = pd.DataFrame(data=amnt_default_rate, columns=['year','month','month_num','default_loans','paid_to_date','total_loans'])
amnt_default_rate_df.to_csv('default_rate')

In [24]:
amnt_default_rate_df

,year,month,month_num,default_loans,paid_to_date,total_loans
0,2007,Dec,12,1,1111.46,2000.0
1,2008,Jan,01,5,32780.39,40675.0
2,2008,Mar,03,2,9229.30,18000.0
3,2008,Apr,04,1,1972.32,10000.0
4,2008,May,05,1,5113.00,6825.0
5,2008,Jul,07,2,8951.83,14500.0
6,2008,Oct,10,3,8346.51,26600.0
7,2008,Nov,11,1,1068.41,3000.0
8,2008,Dec,12,2,15591.94,20000.0
9,2009,Jan,01,6,22325.85,49500.0


In [140]:
test = loans[loans['loan_status'] == "Charged Off" ]
test.to_csv('testing')

In [135]:
loans.loan_status.value_counts()

Current                                                601779
Fully Paid                                             207723
Charged Off                                             45248
Late (31-120 days)                                      11591
Issued                                                   8460
In Grace Period                                          6253
Late (16-30 days)                                        2357
Does not meet the credit policy. Status:Fully Paid       1988
Default                                                  1219
Does not meet the credit policy. Status:Charged Off       761
Name: loan_status, dtype: int64

In [42]:
grade_loans = engine.execute("""
                SELECT
                    year,
                    month,
                    month_num,
                    SUM(loan_amnt) AS value,
                    ROUND(SUM(loan_gmv),2) AS gmv
                FROM
                    loans
                WHERE
                    grade = "G" and loan_status = "Fully Paid"
                GROUP BY
                    1,2
                ORDER BY
                    1,3""").fetchall()
grade_loans_df = pd.DataFrame(data=grade_loans, columns=['year','month','month_num','value','gmv'])
grade_loans_df.to_csv('grade_loans')

In [71]:
loans.shape

(887379, 84)

In [70]:
loan_lending = pd.read_csv('LoanStats3a.csv')
loan_lending.shape

(42539, 1)